### Aspect Orientation Programming

* Normally a function (should) have one core responsibility
* In a business use case we have many different functions like that

#### Use case ---> An eCommerce Application

##### Important Functions

* search_products()
* place_order()
* get_order_list()
* login()

##### Additional Redundant Functions

* we will have some common (redundant) tasks across these functions
* example
    * open database connections
    * handle exceptions
    * measure performance
        * we may want to see how much time some functions take
            * search_products
            * get_order_list
    * handle authentication
        * eg.
            * few functions should work only if user is authenticated
                * place_order
                * get_order list


##### Problem #1 code scattering

* same side asepects need to be applied in multiple functions
* example
    * authentication is applied to
        * place_order
        * get_order_list
    * performance montior is applied to
        * search_products
        * get_order_list

    * exception handling is applied to
        * all most all functionality

* Same logic written again and again
* If the logic changes we need to change everywehere


##### Problem #2 code tangling

* each core function performs multiple side jobs apart from core logic
* example
    * get_order_list
        * core logic
            * search_orders
        * side jobs
            * verify authentication
            * handle exception
            * measure performance

* each function has multiple logic
    * makes code less redable
* when we add new concern it may break old logic.
            
## Let us see the problem in code.

##### Let us create an authenticator
* it is a singleton model
* every one shoudld use same instance of autneticator

In [17]:
class InvalidCredentialsException(Exception):
    pass

class SimpleAuthenticator:
    
    def __init__(self):
        self._user=None # no user logged in

    def is_authenticated(self):
        return self._user is not None
    
    def login(self,user,password):
        users=dict(vivek='p@ss', sanjay='p@ss')
        if user in users:
            _pass=users[user]
            if _pass==password:
                self._user=user
                return True
            
        self._user=None
        raise Exception('Invalid Credentials')
    
    def logout(self):
        self._user=None

SimpleAuthenticator._instance=SimpleAuthenticator()


In [12]:
SimpleAuthenticator._instance.login('vivek','p@ss')

True

In [13]:
SimpleAuthenticator._instance.is_authenticated()

True

In [14]:
SimpleAuthenticator._instance.logout()

In [15]:
SimpleAuthenticator._instance.is_authenticated()

False

In [16]:
SimpleAuthenticator._instance.login('prabhat','p@ss')

Exception: Invalid Credentials

#### Let us now write our business functions

In [17]:
import time

def search_products():
    #performance monitoring
    start=time.time()
    #core business logic
    result=[]

    #exception handling logic
    try: 
        #core business logic
        print('searching products')
        
        for i in range(10):
            time.sleep(0.75)
            result.append(f'Product#{i+1}')
        print('searching products done')

    except Exception as e:
        print(f'error occured :{e}')
    #performance monitoring
    end=time.time()
    print(f'Total Time taken is {end-start}')
    #core business logic
    return result

def get_order_list():
    #performance monitoring
    start=time.time()
    #core business logic

    #authentication validation logic
    if not SimpleAuthenticator._instance.is_authenticated():
        raise Exception('User is not authenticated')

    result=[]

    #exception handling logic
    try: 
        #core business logic
        print('searching products')
        
        for i in range(10):
            time.sleep(0.5)
            result.append(f'Order#{i+1}')
        print('searching products done')
    except Exception as e:
        print(f'error occured :{e}')

    #performance monitoring
    end=time.time()
    print(f'Total Time taken is {end-start}')
    #core business logic
    return result       
    


In [18]:
search_products()

searching products
searching products done
Total Time taken is 7.504557132720947


['Product#1',
 'Product#2',
 'Product#3',
 'Product#4',
 'Product#5',
 'Product#6',
 'Product#7',
 'Product#8',
 'Product#9',
 'Product#10']

#### A generic Performance Monitoring Logic

* can we  write a generic function to monitor performance of any function?
* Simple Performance monitoring had following steps
    1. store sthe start timer
    2. let the business logic run
    3. store the end time
    4. print the log
    
* now we can pass the business logic as call back to performance monitoring 

In [19]:
def find_performance(fn):
    start=time.time()
    fn()
    end=time.time()
    print(f'Total Time taken is {end-start} seconds')

In [22]:
def place_order():
    print('placing order...')
    time.sleep(0.4)
    print('order placed')

In [23]:
#by default placing order will not give you your performance
place_order()

placing order...
order placed


### Measuring the Performance of a function

In [26]:
find_performance(place_order)

placing order...
order placed
Total Time taken is 0.40082502365112305 seconds


#### Problem

* user will not want to wrap their call to place_order in find_performance
    * what if we have optimized and don't want to find_performance anymore?
* user will like to call their function normally


#### Let us comibine call back and closure together

* here we will create a wrapper that will 
    * wrap the core logic and additional concern together
* this wrapper can be considered as a **enchanced version** of my core logic

* the wrapper can be defined using a inner function
* the outer function can help us simply the wrapping per function


#### Code walkthrough

1. performance function takes **target** as callback
    * this target is my core functionality for which we want to measure performance
2. We define a **inner** wrapper
    * wrapper had two key elements
        1. performance monitoring
            * Line 3,5,6
        2. core logic invoked on line #4
        
8. we are returning wrapper function
    * wrapper functionc can be considered as an enchanced (more featurefull version) of actual target
    * It is still performing the core job
    * but it has additional features of performance monitoring

In [30]:
def performance(target):
    def wrapper():
        start=time.time()
        target()
        end=time.time()
        print(f'Total Time taken by {target.__name__} is {end-start} seconds')

    return wrapper

#### Let us see how this thing works

In [31]:
#default place_order doesn't measure performance
place_order()

placing order...
order placed


In [33]:
### But we can create an enhanced version of place_order

place_order_p = performance(place_order)

### now we can use place_order_p instead of place_order whereever we need

place_order_p()

### place_order_p can still place the order. but can also give you time taken


placing order...
order placed
Total Time taken by place_order is 0.4007441997528076 seconds


#### What if we always want to measure performance for place_order

* we may have used place_order() had many places
* now we may have to replace place_order() with place_order_p() everywhere
* this is undeseriable

##### We can replace the reference to the core function with wrapped_logic

In [36]:
def long_running_task():
    print('long running task started')
    time.sleep(2.5)
    print('long running task finished')

#replace original function with wrapped function
long_running_task = performance(long_running_task)

#now when user calls my core function, they actually call this wrapped function


In [37]:
long_running_task()

long running task started
long running task finished
Total Time taken by long_running_task is 2.5009372234344482 seconds


#### Important Note

* here long_running_task is not actually the long_running_task
* it is the wrapper function that **performance()** returned.

##### Can you provde that that long_running_task is not actually long_running_task

* just check the function name

In [38]:
long_running_task.__name__

'wrapper'

### Decorator Design Pattern

* it is a standard design pattern to enchance the functionality using wrappers
* Currently we have achieved he result using  a combination of callback closure

### Python Decorator syntax.

* in the previous example, we wrapped the elements manually

```python
def some_function():
    # some business logic here

some_function= decorate_with_new_function(some_function)
```

* This thing can done by using a smart syntax

```python
@decorate_with_new_function
def some_function():
    # some business logic here
```

In [39]:
@performance
def long_running_task2():
    '''
    This is the long running task 2
    And it has been automatically 
    Decorated with performance
    
    '''
    print('long running task 2 started')
    time.sleep(4.5)
    print('long running task 2 finished')


#### The above code is exactly same as
```python
def long_running_task2():
    pass

long_running_task2 = performance(long_running_task2)
```


In [40]:
long_running_task2()

long running task 2 started
long running task 2 finished
Total Time taken by long_running_task2 is 4.500742673873901 seconds


#### we can see the functions internal name is different

In [41]:
long_running_task2.__name__

'wrapper'

#### We can get the help on this function right now
* the actual function is now hidden
* wrapper doesn't have same docstring

In [42]:
help(long_running_task2)

Help on function wrapper in module __main__:

wrapper()



#### How can we fix this?

* we can copy the name and docstring from original function to wrapper

In [43]:
def performance(target):
    def wrapper():
        start=time.time()
        target()
        end=time.time()
        print(f'Total Time taken by {target.__name__} is {end-start} seconds')

    wrapper.__name__=target.__name__
    wrapper.__doc__=target.__doc__
    return wrapper

In [44]:
@performance
def long_running_task2():
    '''
    This is the long running task 2
    And it has been automatically 
    Decorated with performance
    
    '''
    print('long running task 2 started')
    time.sleep(4.5)
    print('long running task 2 finished')


#### Now we will not be able to see this function different from orginal

In [45]:
long_running_task2()

long running task 2 started
long running task 2 finished
Total Time taken by long_running_task2 is 4.50159478187561 seconds


In [46]:
long_running_task2.__name__

'long_running_task2'

In [47]:
help(long_running_task2)

Help on function long_running_task2 in module __main__:

long_running_task2()
    This is the long running task 2
    And it has been automatically
    Decorated with performance



### This code however will fail if function takes and argument or returns an argument

* consider our current decorator
---
```python
def performance(target):
    def wrapper():
        start=time.time()
        target()
        end=time.time()
        print(f'Total Time taken by {target.__name__} is {end-start} seconds')

    wrapper.__name__=target.__name__
    wrapper.__doc__=target.__doc__
    return wrapper
```
---

* now consider a decorated function
---
```python
@performance
def sum(x,y):
    return x+y
```
---

* after decorating any function, what we really get is **wrapper**

* when user will call sum(2,3), they are passing two parameter
    * but wraper doesn't take any parameer. It will result in error

* wrapper doesn't return a result
    * it will always return None



In [48]:
@performance
def plus(x,y):
    return x+y

In [49]:
plus(2,3)

TypeError: performance.<locals>.wrapper() takes 0 positional arguments but 2 were given

#### How to fix this problem

1. We must pass the same parameters to the wrapper that actual target will take
2. We must return the result returned by actual target


#### Attempt #1

In [50]:
def performance(target):
    def wrapper(x,y):
        start=time.time()
        result=target(x,y)
        end=time.time()
        print(f'Total Time taken by {target.__name__} is {end-start} seconds')
        return result
    wrapper.__name__=target.__name__
    wrapper.__doc__=target.__doc__
    return wrapper

In [51]:
@performance
def sum(x,y):
    return x+y

In [52]:
sum(2,3)

Total Time taken by sum is 0.0 seconds


5

#### Problem
* Now the problem is it works with only those targets that take exactly two parameters
* it will not work if there is parameter mistmatch


In [53]:
@performance
def inverse(x):
    return 1/x

inverse(5)

TypeError: performance.<locals>.wrapper() missing 1 required positional argument: 'y'

#### A Generic Decorator

* Since we don't know what parameters user may pass, to make it generic we should pass
    * \*args to handle all positional parameters
    * \*\*kwargs to handle all keyword parameters

* Generic design of a decorator

```python
def some_decorator( target ):
    def wrapper( *args, **kwargs ):
        #1. do some additional work here, if needed
        
        #2 call the actual function
        result = target( *args, **kwargs )
        
        #3. do some additional work here, if needed

        #4. return the result        
        return result
    #apply name and docs
    wrapper.__name__=target.__name__
    wrapper.__doc__=target.__doc__
    return wrapper
```

* We may also additionally handle exceptions if needed.

* we have a special built-in decorator to handle

```python
#apply name and docs
wrapper.__name__=target.__name__
wrapper.__doc__=target.__doc__
```

* we can replace it by adding **@wraps** decorator over wrapper function
    * this decorator is present in **functools** module


#### Final Version of Few Decorators


##### performance monitor


In [8]:
import time
from functools import wraps
def performance(target):
    
    def wrapper( *args, **kwargs ):
        #1. do some additional work here, if needed
        start=time.time()
        try:
            #2 call the actual function
            result = target( *args, **kwargs )
            return result
        finally:            
            #3. do some additional work here, if needed
            end=time.time()
            print(f'Total Time taken by {target.__name__}({args},{kwargs}) is {end-start} seconds')

    #wrapper.__name__=target.__name__
    #wrapper.__doc__=target.__doc__

    return wrapper

##### 2. authenicated

* rejects the call if you are not authenicated

In [9]:
from functools import wraps

def authenicated(target):
    
    def wrapper( *args, **kwargs ):
        #1. do some additional work here, if needed
        if not SimpleAuthenticator._instance.is_authenticated():
            raise InvalidCredentialsException(f'Invalid Credentials')
        
        #2 call the actual function
        return target( *args, **kwargs )
    
    return wrapper
        

#### Additional Parameters to decorator

* sometimes we want to provide addtional information while decorating a code.
* Example 

##### We want to add simulated delay (for testing purposes) to a code

* these delay should be user defined
* example
    * long running task#1 should delay 2 seconds
    * long running task#2 should delay 4 seconds

* now decorator can't hard code this value

* We want to do something like this

---
```python
@delay(2)
def task1():
    pass

@delay(3)
def task2():
    pass
```
---

* To do this we need to apply one more wrapper

* now we will have three function.
    * parameterized_decorator(extra_parameter) --> target_wrapper(target) -->  wrapper(*args,**kwargs)

---
```python
def my_decorator(extra_parameter):
    def target_wrapper( target):
        @wraps
        def wrapper( *args, **kwargs ):
            #your decorator logic
            result = wrapper(*args, **kwargs)
            return result
        return wrapper
    return target_wrapper
```
---



#### Delay decorator


In [10]:
def delay(seconds):
    def target_wrapper(target):
        #@wraps
        def wrapper(*args, **kwargs):
            #decorator logic
            time.sleep(seconds)
            #actual target call
            return target(*args, **kwargs)

        return wrapper

    return target_wrapper

#### a logger decorator


In [11]:
def log_call(target):
    #@wraps(target)
    def wrapper(*args, **kwargs):
        print(f'Calling {target.__name__}({args},{kwargs})')
        result = target(*args, **kwargs)
        print(f'Finsihed {target.__name__}({args},{kwargs})')
        return result
    return wrapper

#### Multiple Decoration

* we can decorate same function mulitple decorators.
* each will be wrapping one by one in the order they are applied.

```python
@decorator1
@decorator2
@decorator3
def fn():
    pass

```
* is same as

```python
def fn():
    pass

fn = decorator1(decorator2(decorator3(fn)))
```

#### Back to ecommerce application

In [12]:
def search_products(qry):
    result=[]
    for i in range(1,11):
        result.append(f'Product {i}')
        time.sleep(0.5)
    return result

In [13]:
search_products('mobile')

['Product 1',
 'Product 2',
 'Product 3',
 'Product 4',
 'Product 5',
 'Product 6',
 'Product 7',
 'Product 8',
 'Product 9',
 'Product 10']

#### decorated versions

In [14]:

@performance
def search_products(qry):
    result=[]
    for i in range(1,11):
        result.append(f'{qry} {i}')
        time.sleep(0.5)
    return result


@performance
@delay(3)
@authenicated
def search_orders(qry):
    result=[]
    for i in range(1,5):
        result.append(f'Order {qry} {i}')
        
    return result

In [15]:
search_products("Samsung Mobile")

Total Time taken by search_products(('Samsung Mobile',),{}) is 5.00384521484375 seconds


['Samsung Mobile 1',
 'Samsung Mobile 2',
 'Samsung Mobile 3',
 'Samsung Mobile 4',
 'Samsung Mobile 5',
 'Samsung Mobile 6',
 'Samsung Mobile 7',
 'Samsung Mobile 8',
 'Samsung Mobile 9',
 'Samsung Mobile 10']

In [18]:
search_orders("Vivek")

Total Time taken by wrapper(('Vivek',),{}) is 3.000960350036621 seconds


InvalidCredentialsException: Invalid Credentials

In [19]:
SimpleAuthenticator._instance.login('vivek','p@ss')

True

In [20]:
search_orders('Vivek')

Total Time taken by wrapper(('Vivek',),{}) is 3.000385284423828 seconds


['Order Vivek 1', 'Order Vivek 2', 'Order Vivek 3', 'Order Vivek 4']

#### Handle exception decorator

* we may want to long and return a default value in case of an error rather than throwing it

In [23]:
def supress_exception(ExceptionType=Exception, error_result=[]):
    def target_wrapper(target):
        def wrapper(*args, **kwargs):
            try:
                return target(*args, **kwargs)
            
            except ExceptionType as e:
                print(f'LOG: Error {e}')
                return error_result
            
        return wrapper
    return target_wrapper

In [24]:
@performance
@delay(3)
@supress_exception(InvalidCredentialsException, [])
@authenicated
def search_orders(qry):
    result=[]
    for i in range(1,5):
        result.append(f'Order {qry} {i}')
        
    return result

In [25]:
search_orders('Vivek')

Total Time taken by wrapper(('Vivek',),{}) is 3.00089955329895 seconds


['Order Vivek 1', 'Order Vivek 2', 'Order Vivek 3', 'Order Vivek 4']

In [26]:
SimpleAuthenticator._instance.logout()

In [27]:
search_orders('Vivek')

LOG: Error Invalid Credentials
Total Time taken by wrapper(('Vivek',),{}) is 3.000981330871582 seconds


[]

### A Class Decorator

* sometimes we may want to create a class Decorator to add functionalities in a class
* let us say one common function we need in every class is \_\_str\_\_
* by default str returns a useless piece of information
* I want my \_\_str\_\_ to return all properties and their value

```python
book=Book('The Accursed God', 'Vivek Dutta Mishra', 399)
print(book) # Book[ title: 'The Accursed God', author: 'Vivek Dutta Mishra, price: 399] 
```

* we may write a \_\_str\_\_ function to do this in every class. 
    * but that would be redundant.

* we can handle this using a decorator

#### Phase#1 How to find all properties prsent in a object?

* every object has a special member called \_\_dict\_\_.
* It includes all defined proeprties and methods 

In [31]:
class Book:
    def __init__(self,title,author,price):
        self.title=title
        self.author=author
        self.price=price
    

In [32]:
b=Book('The Accursed God','Vivek Dutta Mishra',399)
print(b)

In [33]:
b.__dict__

{'title': 'The Accursed God', 'author': 'Vivek Dutta Mishra', 'price': 399}

#### we can make a pretty string builder

In [34]:
def pretty_string_builder(obj):
    _str= type(obj).__name__
    _str+= str(obj.__dict__)
    return _str

In [35]:
print(b)
print(pretty_string_builder(b))

Book{'title': 'The Accursed God', 'author': 'Vivek Dutta Mishra', 'price': 399}


#### Phase #2 Write a class decorator to add \_\_str\_\_ method to the object

* A class decorator is exactly like a function decorator
* both class and functions are object
* it simply taks class as target

In [38]:

def presentable(cls): # we will decorate a 
    def fn(obj):
        return type(obj).__name__ + str(obj.__dict__)
    
    cls.__str__ = fn
    return cls
        

In [39]:
@presentable
class Book:
    def __init__(self,title,author,price):
        self.title=title
        self.author=author
        self.price=price

In [40]:
b=Book('The Accursed God','Vivek Dutta Mishra',399)
print(b)

Book{'title': 'The Accursed God', 'author': 'Vivek Dutta Mishra', 'price': 399}


In [41]:
@presentable
class Author:
    def __init__(self,name,biography):
        self.name=name
        self.biography=biography

In [42]:
a= Author('Mahatma Gandhi', 'The father of the nation India')
print(a)

Author{'name': 'Mahatma Gandhi', 'biography': 'The father of the nation India'}


In [43]:
a.photo="gandhi.jpg"
print(a)

Author{'name': 'Mahatma Gandhi', 'biography': 'The father of the nation India', 'photo': 'gandhi.jpg'}


In [44]:
@presentable
class Point:
    def __init__(self,x,y):
        self.x=x
        self.y=y
        

In [45]:
points=[ Point(3,4), Point(1,1), Point(2,0)]

for p in points:
    print(p)

Point{'x': 3, 'y': 4}
Point{'x': 1, 'y': 1}
Point{'x': 2, 'y': 0}


### Singleton decorat

* sometimes we want a class to have a single instance.
* we want to avoid creating a second object if the first object exists
* we want to achieve this using a decorator

#### How it will work

* it will define it's own 
    * __init__
        * to check if the object is already created.
    * instance

* normally 
    * for any singlton get also have a instance() function to return it's instance
    * there __init__ should be parameterless

In [46]:
def Singleton(cls):
    cls.__instance=None
    original_init= cls.__init__

    def new_init(self):
        if cls.__instance is not None:
            raise Exception("Single doesn't allow a second object")
        else:
            original_init(self)
            cls.__instance=self
            

    def instance():
        if cls.__instance is None:
            cls.__instance=cls()
        return cls.__instance
    
    cls.__init__=new_init
    cls.instance=instance

    return cls


In [49]:
@Singleton
class AuthManager:
    def __init__(self):
        self._users=dict(vivek='p@ss', sanjay="p@ss")
        self._user=None

    def authenticate(self,username,password):
        if username in self._users and self._users[username]==password:
            self._user=username
            return True
        else:
            self._user=None
            raise InvalidCredentialsException("Invalid Credentials")
        
    def loutout(self): self._user=None
    def is_authenticated(self): return self._user is not None

In [50]:
a1 = AuthManager()

In [51]:
a2= AuthManager()

Exception: Single doesn't allow a second object

In [52]:
AuthManager.instance().authenticate('vivek','p@ss')

True

In [63]:
@Singleton
class PrintManager:
    pass

In [64]:
p1= PrintManager.instance()
p2= PrintManager.instance()

print(p1 is p2)

True


In [65]:
print(dir(PrintManager))

['__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getstate__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__instance', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', 'instance']


In [66]:
p2=PrintManager()

Exception: Single doesn't allow a second object